# Spectral Resolution & Efficiency vs Grism Resolvance Trade
Tradeoff between spectral resolution and effiency vs grism resolvance.

In [6]:
# external
import astropy.units as unit
import numpy as np
import pandas as pd
import plot
from IPython.display import display

# project
from payload_designer import components, systems
from payload_designer.libs import utillib

## Setup
Run this section onwards when a parameter is updated.

### Parameters

In [7]:
# constants
altitude = 550 * unit.km
diameter = 100 * unit.mm
skew_angle = np.array([0, 0]) * unit.deg
slit_size = np.array([3, 1]) * unit.mm
upper_wavelength = 1700 * unit.nm
lower_wavelength = 900 * unit.nm


# variables
focal_length = np.arange(start=10, stop=300, step=10) * unit.mm

# vectorization
parameter_space_shape = (focal_length.size, slit_size.size)
focal_length = utillib.orient_and_broadcast(
    a=focal_length, dim=0, shape=parameter_space_shape
)
# slit_size = utillib.orient_and_broadcast(a=slit_size, dim=1, shape=parameter_space_shape)

### Instantiation

In [8]:
# components
sensor = components.sensors.TauSWIR()
foreoptic = components.foreoptics.Foreoptic(
    diameter=diameter, focal_length=focal_length
)
slit = components.masks.RectSlit(size=slit_size)

# systems
payload = systems.payloads.HyperspectralImager(sensor=sensor, foreoptic=foreoptic, slit=slit)

## Pipeline

In [9]:
sensor_spectral_resolution = payload.get_spectral_resolution(
    upper_wavelength=upper_wavelength, lower_wavelength=lower_wavelength
)
print(f"Sensor-limited spectral resolution shape: {sensor_spectral_resolution}")

#upper_wavelength, lower_wavelength, n_biny, n_px_y

Sensor-limited spectral resolution shape: 1.5625 nm / pix


## Plots

In [10]:
data = {
    f"Focal Length [{focal_length.unit}]": focal_length[:, 0],
    f"Spatial Resolution (across-track) [{sensor_spatial_resolution.unit}]": sensor_spatial_resolution[
        :, 0
    ],
    f"Spatial Resolution (along-track) [{sensor_spatial_resolution.unit}]": sensor_spatial_resolution[
        :, 1
    ],
    f"Swath (across-track) [{swath.unit}]": swath[:, 0],
    f"Swath (along-track) [{swath.unit}]": swath[:, 1],
}

df = pd.DataFrame.from_dict(data=data)
graph = plot.line(
    df=df,
    x=f"Focal Length [{focal_length.unit}]",
    y=[
        f"Spatial Resolution (across-track) [{sensor_spatial_resolution.unit}]",
        f"Spatial Resolution (along-track) [{sensor_spatial_resolution.unit}]",
        f"Swath (across-track) [{swath.unit}]",
        f"Swath (along-track) [{swath.unit}]",
    ],
    title="FINCH Eye Spatial Resolution & Swath vs. Focal Length",
    dark=True,
)

graph.show()
display(df)

NameError: name 'sensor_spatial_resolution' is not defined